In [ ]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [ ]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)



In [ ]:
# Get only the WRR items
wrr_df = mss_df.loc[mss_df['UniqueID'].str.contains("WRR")].copy()

In [ ]:
# Make rough draft of the client table
client_df = wrr_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

# fix up Reporting Period column
client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
# Fix up age column
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
# fix up future gt column
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
# fix up Tract and Stand columns
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
# fix up client gt column
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

# fix up harvest instake columns
cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


# Normalize species

species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))

# Normalize Origin

origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))



In [ ]:
"""
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
client_df['id'] = client_df['uniqueid']
client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]
"""

In [ ]:
ga_intake_df = wrr_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [ ]:
status_df = wrr_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [ ]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [ ]:
wrr_inprog = wrr_df[wrr_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
wrr_inprog['Tract'] = wrr_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['CMP'] = wrr_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['Stand'] = wrr_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['TCS'] = wrr_inprog['Tract'] + "_" + wrr_inprog['CMP'] + "_" + wrr_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = wrr_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = wrr_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)

# looking for TCS combinations in previous ticketids        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
if len(ticket_id_dict) > 0:
    for k, v in ticket_id_dict.items():
        trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k
else:
    trial_df['otherid'] = None
    

def final_id(row):
    if row['otherid'] != None:
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(wrr_df['UniqueID'], wrr_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WRR"), 'TicketID'] = new_tcs_intake['old_ticketids']


ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

Getting the mill tickets

In [ ]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WRR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WRR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

# big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
# big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
# big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
# big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "rp" in str(x).lower():
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'], errors='coerce')

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

In [ ]:
mill_df.head()

In [ ]:
# Getting the old Harvest TicketIDs
harvested_tons_oldTID = mill_df[mill_df['Reporting Period'] == 7].groupby("TicketID")['Tons (U.S)'].sum().reset_index()

In [ ]:
harvested_tons_oldTID['Tons (U.S)'].sum()

Let's get new and old TIDs that are currently in process

In [ ]:
# Getting harvest ids that were in RP 7 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [ ]:
# Let's see how many tons are under old TIDs of activities in question
harvested_tons_oldTID[harvested_tons_oldTID['TicketID'].isin(harvest_old_ticketid)]['Tons (U.S)'].sum()

In [ ]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [ ]:
# Let's see what is in old_ticketid that is not in new_ticketIDs
[item for item in harvest_old_ticketid if item not in new_ticketIDs]

In [ ]:
for id in harvest_old_ticketid:
    old_unique = wrr_df[wrr_df['TicketID'] == id]['UniqueID'].unique().tolist()
    new_unique = ticketid_table[ticketid_table['TicketID'] == id]['UniqueID'].unique().tolist()
    inold_notinew = [item for item in old_unique if item not in new_unique]
    if len(inold_notinew) > 0:
        print(id, old_unique)

Ok, so some old ticketids don't line up with the new ticket ids

Let's update the mill compiled file and replace the ticket ids

In [ ]:
mill_df[mill_df['TicketID'].isin(['WRR231107CC02', 'WRR231107TT01'])]['TicketID'].unique()

In [ ]:
mill_df[mill_df['TicketID'].str.contains("WRR231107CC02")]['TicketID'].unique().tolist()

In [ ]:
# Game plan is to get the old uniqueid -> ticketid connection and then find the new ticketid.
# Then replace all ticketids in the mill df to the new using a map

wrr_df_uid_tid = wrr_df[['UniqueID', 'TicketID']].copy()
wrr_df_uid_tid = pd.merge(wrr_df_uid_tid, ticketid_table, on='UniqueID', how="left")

In [ ]:
ticketid_table[ticketid_table['UniqueID'].isin(['WRR231107CC02', 'WRR231107TT01'])]

In [ ]:
wrr_df_uid_tid[wrr_df_uid_tid['TicketID_y'].str.contains('X')]
wrr_df_tids = wrr_df_uid_tid[['TicketID_x', 'TicketID_y']].copy()

In [ ]:
wrr_df_tids.drop_duplicates(inplace=True)
wrr_tid_mapper = dict(zip(wrr_df_tids['TicketID_x'], wrr_df_tids['TicketID_y']))

In [ ]:
mill_df['new_ticketid'] = mill_df['TicketID'].map(wrr_tid_mapper)

In [ ]:
new_ticket_harvested = mill_df.groupby('new_ticketid')['Tons (U.S)'].sum().reset_index()
new_ticket_harvested['Tons (U.S)'].sum()

In [ ]:
mill_df['Tons (U.S)'].sum()

In [ ]:
mill_df[mill_df['new_ticketid'].isna()]

In [ ]:
# There was an issue where WRR230220HD01 was mislabeled WRR230220CC01 in the RP7 update compiled. I fixed that at the source on 2024.11.04

In [ ]:
# harvested_tons_oldTID.to_excel("harvested_tons_compare_old.xlsx", sheet_name='oldTID')
# new_ticket_harvested.to_excel("harvested_tons_compare_new.xlsx", sheet_name='newTID')

In [ ]:
trial_expected = pd.merge(ticketid_table, wrr_df[['UniqueID', 'Current RP Expected GT', 'Status', 'Future GT']], on='UniqueID', how='left')
trial_expected = pd.merge(trial_expected, client_df[['UniqueID', 'Reporting Period']], on='UniqueID', how='left')
wrr_rp7_expected = trial_expected[(trial_expected['Reporting Period'] == 7) & trial_expected['Status'].isin(['IN PROCESS', 'OPEN', 'POST'])].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
wrr_rp7_expected = pd.merge(wrr_rp7_expected, harvested_tons_oldTID, on='TicketID', how='left')
wrr_rp7_expected['Tons (U.S)'] = pd.to_numeric(wrr_rp7_expected['Tons (U.S)'], errors='coerce', downcast='float')
wrr_rp7_expected['Current RP Expected GT'] = pd.to_numeric(wrr_rp7_expected['Current RP Expected GT'], errors='coerce', downcast='float')
wrr_rp7_expected['Tons (U.S)'].fillna(0, inplace=True)
wrr_rp7_expected['Current RP Expected GT'].fillna(0, inplace=True)
wrr_rp7_expected['Remaining'] = wrr_rp7_expected['Current RP Expected GT'] - wrr_rp7_expected['Tons (U.S)']

In [ ]:
wrr_rp7_expected[wrr_rp7_expected['Remaining'] > 0]

In [ ]:
# wrr_rp7_expected.to_excel("wrr_rp7_accouting.xlsx")

In [ ]:
wrr_rp8_expected = trial_expected[(trial_expected['Reporting Period'] == 8) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
wrr_rp8_expected_1 = trial_expected[(trial_expected['Reporting Period'] == 7) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Future GT'].sum().reset_index()

In [ ]:
wrr_rp8_expected = pd.merge(wrr_rp8_expected, wrr_rp8_expected_1, on='TicketID', how='outer')

In [ ]:
wrr_rp8_expected['Current RP Expected GT'] = pd.to_numeric(wrr_rp8_expected['Current RP Expected GT'], errors='coerce', downcast='float')
wrr_rp8_expected['Future GT'] = pd.to_numeric(wrr_rp8_expected['Future GT'], errors='coerce', downcast='float')
wrr_rp8_expected.fillna(0, inplace=True)
wrr_rp8_expected['RP8_Expected'] = wrr_rp8_expected['Current RP Expected GT'] + wrr_rp8_expected['Future GT']

In [ ]:
wrr_rp8_trial = pd.merge(wrr_rp8_expected, wrr_rp7_expected, on='TicketID', how='left')

In [ ]:
wrr_rp8_trial
wrr_rp8_trial.fillna(0, inplace=True)
wrr_rp8_trial['final_expected'] = wrr_rp8_trial['RP8_Expected'] + wrr_rp8_trial['Remaining']

Nice, this adds up with Rob's file. Lets get the info needed to fill out the Shared Space table

In [ ]:
wrr_ticketid_volume = wrr_rp8_trial[['TicketID', 'final_expected']]

In [ ]:
ticket_mapper = dict(zip(ticketid_table['UniqueID'], ticketid_table['TicketID']))

In [ ]:
wrr_df['new_ticket_id'] = wrr_df['UniqueID'].map(ticket_mapper)

In [ ]:
# wrr_rp8_trial.to_excel('wrr_rp8_investigate_3.xlsx')

In [ ]:
ticketid_table.loc[ticketid_table['TicketID'] == 'WRR231107XXXX']

In [ ]:
# Ok. Let's create the client facing table and get it up on the drive


In [ ]:
client_df.head()

In [ ]:
ga_intake_df.head()

In [ ]:
mill_df.head() # make sure to use the correct ticketid long term

In [ ]:
ticketid_table

In [ ]:
new_tcs_intake

In [ ]:
wrr_inprog[['UniqueID', 'TCS']]

In [ ]:
status_df

In [ ]:
client_table_inprog = status_df[status_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST'])].copy()
client_table_inprog = pd.merge(client_table_inprog, wrr_inprog, on='UniqueID', how='left')
client_table_inprog = pd.merge(client_table_inprog, ticketid_table, on='UniqueID', how='left')

In [ ]:
client_table_inprog.columns

In [ ]:
client_table_inprog = client_table_inprog[['UniqueID', 'TCS', 'Status_x', 'Guidance_x', 'TicketID_y']]

In [ ]:
expected_aligned_with_rob = pd.read_excel("./WRR_RP8_InProcess/wrr_rp8_investigate_3.xlsx")
expected_aligned_with_rob = expected_aligned_with_rob[['TicketID', 'Final Values']]
expected_aligned_with_rob['RP8_Expected'] = pd.to_numeric(expected_aligned_with_rob['Final Values'])

In [ ]:
client_table_inprog = pd.merge(client_table_inprog, expected_aligned_with_rob, left_on='TicketID_y', right_on='TicketID', how='left')

In [ ]:
client_table_inprog

In [ ]:
client_final_df = client_table_inprog.groupby('TicketID').agg({'TCS': ", ".join,
                                               'Status_x': ", ".join,
                                               'Guidance_x': ", ".join,
                                               'RP8_Expected': 'first'}).reset_index()

In [ ]:
client_final_df['RP8_Expected'].sum()

In [ ]:
client_final_df

I totally did not make the expected by RP table

I need to get total expected by activity

and

I need to get the expected by RP

Fortunately WRR doesn't have any "Future" rp volumes to worry about - just carry over, and we are only worrying about RP7 -> RP8 for that

and RP8 is figured out already from above.

So, we just need to grab all the previous volumes to build out the individual RP expected volumes.

The trick will be finding the total? Or should we just base it off the total in mss?

In [ ]:
wrr_df[['UniqueID', 'Reporting Period', 'Current RP Expected GT', 'Total GT']]

In [ ]:
wrr_expected_inprog = wrr_df[wrr_df['Reporting Period'].notna()][['UniqueID', 'Reporting Period', 'Current RP Expected GT', 'Total GT']].copy()

In [ ]:
wrr_expected_inprog['Reporting Period'].fillna(0, inplace=True)
wrr_expected_inprog.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = wrr_expected_inprog.loc[pd.to_numeric(wrr_expected_inprog['Reporting Period'], errors='coerce').isnull() & wrr_expected_inprog['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
wrr_expected_inprog['trial'] = wrr_expected_inprog.index.map(s)
wrr_expected_inprog.loc[pd.to_numeric(wrr_expected_inprog['Reporting Period'], errors='coerce').isnull() & wrr_expected_inprog['Reporting Period'].str.contains("RP"), 'Reporting Period'] = wrr_expected_inprog.loc[pd.to_numeric(wrr_expected_inprog['Reporting Period'], errors='coerce').isnull() & wrr_expected_inprog['Reporting Period'].str.contains("RP"), 'trial']
wrr_expected_inprog['Reporting Period'] = pd.to_numeric(wrr_expected_inprog['Reporting Period'], downcast='integer')

In [ ]:
wrr_expected_inprog['Current RP Expected GT'] = pd.to_numeric(wrr_expected_inprog['Current RP Expected GT'], errors='coerce')
wrr_expected_inprog['Total GT'] = pd.to_numeric(wrr_expected_inprog['Total GT'], errors='coerce')

In [ ]:
wrr_expected_inprog = wrr_expected_inprog[((wrr_expected_inprog['Current RP Expected GT'].notna()) | (wrr_expected_inprog['Total GT'].notna())) & (wrr_expected_inprog['Reporting Period'] < 8)]
wrr_expected_inprog

Ok, well.. Looks like it is all RP7 - but that is the data we have.

Time to tie in the TicketID and then groupby!

In [ ]:
wrr_expected_inprog = pd.merge(wrr_expected_inprog, ticketid_table, on='UniqueID', how='left')

In [ ]:
wrr_expected = wrr_expected_inprog.groupby(['TicketID', 'Reporting Period'])['Current RP Expected GT'].sum().reset_index()

In [ ]:
rp8_expect = client_final_df[['TicketID', 'RP8_Expected']].copy()
rp8_expect['Reporting Period'] = 8
rp8_expect['Current RP Expected GT'] = rp8_expect['RP8_Expected']
rp8_expect = rp8_expect[['TicketID', 'Current RP Expected GT', 'Reporting Period']].copy()
wrr_expected =pd.concat([wrr_expected, rp8_expect])

In [ ]:
wrr_expected

What is the aggregation of "total gt" for each new ticketid? 

After some looking - it looks like we don't need to worry about this for the current rp.

In [ ]:
total_expect_inprog = wrr_df[['UniqueID', 'Total GT']].copy()
total_expect_inprog = pd.merge(total_expect_inprog, ticketid_table, on='UniqueID', how='inner')

In [ ]:
total_expect_inprog['Total GT'] = pd.to_numeric(total_expect_inprog['Total GT'], errors='coerce')
total_expect_interest = total_expect_inprog[total_expect_inprog['Total GT'] > 0].copy()
ticket_total = total_expect_interest.groupby('TicketID')['Total GT'].sum().reset_index()

In [ ]:
ticket_total.to_excel('first_ticket_total.xlsx')

Let's get the harvested volumes for RP8 and total then for these magical beasts

In [ ]:
grouped_mill_rp = mill_df.groupby(['new_ticketid', 'Reporting Period'])['Tons (U.S)'].sum().reset_index()
grouped_mill_total = mill_df.groupby('new_ticketid')['Tons (U.S)'].sum().reset_index()

In [ ]:
# able to get either at rp level or total for ticket id
# new_ticket_total = pd.merge(ticket_total, grouped_mill_rp[grouped_mill_rp['Reporting Period'] > 7], left_on='TicketID', right_on='new_ticketid', how='inner')
new_ticket_total = pd.merge(ticket_total, grouped_mill_total, left_on='TicketID', right_on='new_ticketid', how='inner')

In [ ]:
# new_ticket_total.to_excel('total_volume.xlsx')

Let's get the last ticket date

In [ ]:
ticket_recentdate = mill_df.groupby('new_ticketid')['Date'].last().reset_index()
# ticket_recentdate.to_excel('ticketdate.xlsx')

In [ ]:
ticketid_table[ticketid_table['UniqueID'] == 'WRR240101CC01']

In [ ]:
ticketid_table[ticketid_table['TicketID'].str.contains('X')]['TicketID'].unique().tolist()

In [ ]:
ticketid_table[ticketid_table['TicketID'] == 'WRR231107XXXX']

In [ ]:
mill_df[mill_df['TicketID'] == 'WRR231107CC02']

In [ ]:
unique_lst = ticketid_table['UniqueID'].unique().tolist()
len(wrr_df[(wrr_df['UniqueID'].isin(unique_lst)) & (wrr_df['Status'].isin(['IN PROCESS', 'OPEN', 'POST']))]['TicketID'].unique().tolist())

In [ ]:
inprog_wrr_df = wrr_df[wrr_df['Status'].isin(['IN PROCESS', 'POST'])].copy()
inprog_wrr_oldTID = inprog_wrr_df.groupby(['TicketID'])['UniqueID'].apply(list).reset_index()
# inprog_wrr_oldTID.to_excel("old_tid_unique.xlsx")

In [ ]:
ticketid_table[ticketid_table['TicketID'].isin(['WRR230829CCXX',
                                                'WRR230829TH01',
                                                'WRR230829TH02',
                                                'WRR231107CC01',
                                                'WRR231107CC03',
                                                'WRR231107TH01',
                                                'WRR231107XXXX',
                                                'WRR240101TH01',
                                                'WRR240101TT01',
                                                'WRR240101XXXX',
                                                'WRR240404TH01',
                                                'WRR240404TH02',
                                                'WRR240709CCXX',
                                                'WRR240802TH01',
                                                'WRR240802TH03'])]